In [2]:
# 1️⃣ Imports and setup
import sys
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path

# Project root for module imports
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)
print("Project root added to sys.path:", project_root)

# -----------------------------
# MLflow setup (Windows-safe)
# -----------------------------
import mlflow
import mlflow.sklearn

# Use local mlruns folder
mlruns_path = Path(project_root) / "mlruns"
mlflow_tracking_uri = mlruns_path.as_uri()  # Converts to file:///D:/... format correctly
mlflow.set_tracking_uri(mlflow_tracking_uri)

# Create or set experiment (tracking only; registry won't work locally)
mlflow.set_experiment("Credit_Risk_Modeling_Task5")

print("MLflow tracking URI:", mlflow.get_tracking_uri())
print("MLflow experiment set successfully!")

# -----------------------------
# Module imports from src
# -----------------------------
from src.data_processing import (
    load_data,
    get_numeric_columns,
    get_categorical_columns,
    data_overview,
    summarize_numeric,
    summarize_categorical,
    missing_value_report,
    plot_correlation_matrix,
    top_correlations,
    plot_numeric_histograms,
    plot_numeric_density,
    plot_numeric_boxplots,
    plot_categorical_distribution,
    detect_outliers
)

# -----------------------------
# Data path for processed dataset
# -----------------------------
data_path = Path(project_root) / "data" / "processed" / "rfm_model_ready.csv"
print("Data path set to:", data_path)

Project root added to sys.path: D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4


D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:177: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri, store_uri)
D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\scorecardpy\germancredit.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


MLflow tracking URI: file:///D:/Personal/KAIM-10%20Academy/Week%204/Project/Credit-Risk-Modeling-Week%204/mlruns
MLflow experiment set successfully!
Data path set to: D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\data\processed\rfm_model_ready.csv


Data Loading and Splitting

In [37]:
# ============================================
# TASK 5.2: DATA PREPARATION - Train/Test Split
# ============================================

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# -------------------------------
# 1️⃣ Define project root and data path
# -------------------------------
project_root = os.path.abspath("..")  # notebooks/ -> project root
data_file = os.path.join(project_root, "data", "processed", "rfm_model_ready.csv")

# -------------------------------
# 2️⃣ Load processed CSV
# -------------------------------
df = pd.read_csv(data_file)
print(f"Step 1: Data Loaded ✅\nShape: {df.shape}")

# -------------------------------
# 3️⃣ Separate features and target
# -------------------------------
target_col = 'is_high_risk'
X = df.drop(columns=[target_col])
y = df[target_col]
print(f"Step 2: Features and target separated ✅\nX shape: {X.shape}, y shape: {y.shape}")

# -------------------------------
# 4️⃣ Split into train/test sets
# -------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Step 3: Train/Test split ✅\nTrain: {X_train.shape}, Test: {X_test.shape}")

# -------------------------------
# 5️⃣ Define numeric & categorical features
# -------------------------------
num_features = ['Recency', 'Frequency', 'Monetary']
cat_features = ['Cluster']

print(f"Step 4: Features identified ✅")
print(f"Numerical: {num_features}")
print(f"Categorical: {cat_features}")

# -------------------------------
# 6️⃣ Create preprocessor
# -------------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), cat_features)
    ]
)
print("Step 5: Preprocessor created ✅")

# -------------------------------
# 7️⃣ Apply preprocessing
# -------------------------------
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Step 6: Preprocessing applied ✅")
print(f"X_train processed shape: {X_train_processed.shape}")
print(f"X_test processed shape:  {X_test_processed.shape}")

Step 1: Data Loaded ✅
Shape: (3632, 7)
Step 2: Features and target separated ✅
X shape: (3632, 6), y shape: (3632,)
Step 3: Train/Test split ✅
Train: (2905, 6), Test: (727, 6)
Step 4: Features identified ✅
Numerical: ['Recency', 'Frequency', 'Monetary']
Categorical: ['Cluster']
Step 5: Preprocessor created ✅
Step 6: Preprocessing applied ✅
X_train processed shape: (2905, 5)
X_test processed shape:  (727, 5)


TASK 5.2: SETUP FOR REPRODUCIBILITY

In [38]:
# ============================================
# TASK 5.2: SETUP FOR REPRODUCIBILITY
# ============================================

# GLOBAL RANDOM STATE - Set once, use everywhere
RANDOM_STATE = 42  # Task 5.2 requirement: "Ensure reproducibility by setting a random_state"

print(f"🔧 Random State set to: {RANDOM_STATE} (for reproducibility)")
print("=" * 60)

# Example usage in train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# Example usage in models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

logreg = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)

🔧 Random State set to: 42 (for reproducibility)


 TASK 5.3: MODEL SELECTION AND TRAINING

In [39]:
# ============================================
# TASK 5.3: TRAIN ALL 4 MODELS WITH PIPELINES + EVALUATION
# ============================================

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

print("TASK 5.3: Training ALL 4 Models with Pipelines + Evaluation")
print("=" * 60)

# ---------------------------
# 1. Preprocessing pipeline
# ---------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), ['Frequency', 'Monetary'])
    ]
)

# ---------------------------
# 2. Model pipelines
# ---------------------------
pipelines = {
    'Logistic Regression': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            random_state=42, max_iter=1000, class_weight='balanced'
        ))
    ]),
    
    'Decision Tree': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(
            random_state=42, class_weight='balanced', max_depth=5
        ))
    ]),
    
    'Random Forest': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            random_state=42, n_estimators=100, class_weight='balanced'
        ))
    ]),
    
    'Gradient Boosting': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(
            random_state=42, n_estimators=100, learning_rate=0.1
        ))
    ])
}

# ---------------------------
# 3. Train & Evaluate
# ---------------------------
trained_pipelines = {}

for name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    trained_pipelines[name] = pipeline
    
    # Predictions
    y_pred = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)[:, 1] if hasattr(pipeline.named_steps['classifier'], "predict_proba") else None
    
    # Metrics
    test_acc = pipeline.score(X_test, y_test)
    roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else "N/A"
    
    print(f"\n=== {name} ===")
    print(f"Testing Accuracy: {test_acc:.3f}")
    print(f"ROC-AUC: {roc_auc}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=3))

print(f"\n✅ Task 5.3 COMPLETE: Trained {len(trained_pipelines)} models with pipelines")
print("   ✓ Logistic Regression ✓ Decision Tree ✓ Random Forest ✓ Gradient Boosting")
print("=" * 60)


TASK 5.3: Training ALL 4 Models with Pipelines + Evaluation

=== Logistic Regression ===
Testing Accuracy: 0.623
ROC-AUC: 0.7254067254067255
Classification Report:
              precision    recall  f1-score   support

           0      0.867     0.489     0.626       468
           1      0.484     0.865     0.620       259

    accuracy                          0.623       727
   macro avg      0.676     0.677     0.623       727
weighted avg      0.731     0.623     0.624       727


=== Decision Tree ===
Testing Accuracy: 0.685
ROC-AUC: 0.7512952512952512
Classification Report:
              precision    recall  f1-score   support

           0      0.846     0.624     0.718       468
           1      0.539     0.795     0.643       259

    accuracy                          0.685       727
   macro avg      0.693     0.710     0.681       727
weighted avg      0.737     0.685     0.691       727


=== Random Forest ===
Testing Accuracy: 0.671
ROC-AUC: 0.7401742401742402
Classific

Setup MLflow Experiment

In [40]:
import mlflow

EXPERIMENT_NAME = "Credit_Risk_Modeling_Task5"

mlruns_path = "./mlruns"  # relative path (no file:// needed)
mlflow.set_tracking_uri(mlruns_path)
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"MLflow experiment ready: {EXPERIMENT_NAME}")

MLflow experiment ready: Credit_Risk_Modeling_Task5


In [43]:
# ============================================
# TASK 5.3: Training ALL 4 Models with Pipelines + Evaluation
# ============================================

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Define models
models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Train and evaluate
for name, model in models.items():
    model.fit(X_train_processed, y_train)
    y_pred = model.predict(X_test_processed)
    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, model.predict_proba(X_test_processed)[:,1])
    print(f"\n=== {name} ===")
    print(f"Testing Accuracy: {acc:.3f}")
    print(f"ROC-AUC: {roc:.3f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
print("\n✅ Task 5.3 COMPLETE: Trained 4 models with pipelines")


=== Logistic Regression ===
Testing Accuracy: 0.644
ROC-AUC: 0.512
Classification Report:
              precision    recall  f1-score   support

           0       0.64      1.00      0.78       468
           1       0.00      0.00      0.00       259

    accuracy                           0.64       727
   macro avg       0.32      0.50      0.39       727
weighted avg       0.41      0.64      0.50       727


=== Decision Tree ===
Testing Accuracy: 0.536
ROC-AUC: 0.487
Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.66      0.65       468
           1       0.33      0.31      0.32       259

    accuracy                           0.54       727
   macro avg       0.48      0.48      0.48       727
weighted avg       0.53      0.54      0.53       727



D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zer


=== Random Forest ===
Testing Accuracy: 0.568
ROC-AUC: 0.500
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.72      0.68       468
           1       0.37      0.30      0.33       259

    accuracy                           0.57       727
   macro avg       0.51      0.51      0.51       727
weighted avg       0.55      0.57      0.56       727


=== Gradient Boosting ===
Testing Accuracy: 0.620
ROC-AUC: 0.510
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.94      0.76       468
           1       0.28      0.04      0.07       259

    accuracy                           0.62       727
   macro avg       0.46      0.49      0.42       727
weighted avg       0.51      0.62      0.52       727


✅ Task 5.3 COMPLETE: Trained 4 models with pipelines


TASK 5.3: MLflow Logging of Trained Models

In [45]:
# ============================================
# TASK 5.3: MLflow Logging of Trained Models
# ============================================

import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, roc_auc_score

EXPERIMENT_NAME = "Credit_Risk_Modeling_Task5"
mlflow.set_experiment(EXPERIMENT_NAME)

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Log model
        mlflow.sklearn.log_model(model, name="model_pipeline")
        
        # Predict on test set
        y_pred = model.predict(X_test_processed)
        y_proba = model.predict_proba(X_test_processed)[:,1] if hasattr(model, "predict_proba") else None
        
        # Calculate metrics
        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
        
        # Log metrics
        mlflow.log_metric("accuracy", acc)
        if roc is not None:
            mlflow.log_metric("roc_auc", roc)
        
        # Print summary safely
        roc_str = f"{roc:.3f}" if roc is not None else "N/A"
        print(f"✅ Logged {name} to MLflow with Accuracy={acc:.3f}, ROC-AUC={roc_str}")

✅ Logged Logistic Regression to MLflow with Accuracy=0.644, ROC-AUC=0.512
✅ Logged Decision Tree to MLflow with Accuracy=0.536, ROC-AUC=0.487
✅ Logged Random Forest to MLflow with Accuracy=0.568, ROC-AUC=0.500
✅ Logged Gradient Boosting to MLflow with Accuracy=0.620, ROC-AUC=0.510


Task 5.4 – Hyperparameter Tuning for Credit Risk Models

In [47]:
# ============================================
# TASK 5.4: Hyperparameter Tuning + MLflow Logging
# ============================================

import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

EXPERIMENT_NAME = "Credit_Risk_Modeling_Task5"
mlflow.set_experiment(EXPERIMENT_NAME)

# Assume `models` is a dictionary of your trained/tuned models
# e.g., models = {'Logistic Regression': tuned_lr, ...}

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Log model
        mlflow.sklearn.log_model(model, artifact_path="model_pipeline")
        
        # Predict on test set
        y_pred = model.predict(X_test_processed)
        y_proba = model.predict_proba(X_test_processed)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Calculate metrics
        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
        
        # Log metrics
        mlflow.log_metric("accuracy", acc)
        if roc is not None:
            mlflow.log_metric("roc_auc", roc)
        
        # Safely format ROC-AUC for printing
        roc_str = f"{roc:.3f}" if roc is not None else "N/A"
        print(f"✅ Logged {name} to MLflow with Accuracy={acc:.3f}, ROC-AUC={roc_str}")
        print(classification_report(y_test, y_pred))

2025/12/18 09:15:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\venv\Lib\site-packages\sklearn\metrics\_classification.py:1833: Undefine

✅ Logged Logistic Regression to MLflow with Accuracy=0.644, ROC-AUC=0.512
              precision    recall  f1-score   support

           0       0.64      1.00      0.78       468
           1       0.00      0.00      0.00       259

    accuracy                           0.64       727
   macro avg       0.32      0.50      0.39       727
weighted avg       0.41      0.64      0.50       727



2025/12/18 09:15:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Logged Decision Tree to MLflow with Accuracy=0.536, ROC-AUC=0.487
              precision    recall  f1-score   support

           0       0.63      0.66      0.65       468
           1       0.33      0.31      0.32       259

    accuracy                           0.54       727
   macro avg       0.48      0.48      0.48       727
weighted avg       0.53      0.54      0.53       727



2025/12/18 09:15:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Logged Random Forest to MLflow with Accuracy=0.568, ROC-AUC=0.500
              precision    recall  f1-score   support

           0       0.65      0.72      0.68       468
           1       0.37      0.30      0.33       259

    accuracy                           0.57       727
   macro avg       0.51      0.51      0.51       727
weighted avg       0.55      0.57      0.56       727

✅ Logged Gradient Boosting to MLflow with Accuracy=0.620, ROC-AUC=0.510
              precision    recall  f1-score   support

           0       0.64      0.94      0.76       468
           1       0.28      0.04      0.07       259

    accuracy                           0.62       727
   macro avg       0.46      0.49      0.42       727
weighted avg       0.51      0.62      0.52       727



Save the best models from Grid Search

In [48]:
# ============================================
# TASK 5.5: Save Best Models Locally
# ============================================

import joblib
import os

# Create a folder to save the models
models_dir = os.path.join(project_root, "saved_models")
os.makedirs(models_dir, exist_ok=True)

# Save the best models from GridSearchCV
best_models = {
    "Gradient_Boosting": tuned_gb_model,
    "Random_Forest": tuned_rf_model
}

for name, model in best_models.items():
    file_path = os.path.join(models_dir, f"{name}_best_model.pkl")
    joblib.dump(model, file_path)
    print(f"✅ Saved {name} model at {file_path}")

✅ Saved Gradient_Boosting model at D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\saved_models\Gradient_Boosting_best_model.pkl
✅ Saved Random_Forest model at D:\Personal\KAIM-10 Academy\Week 4\Project\Credit-Risk-Modeling-Week 4\saved_models\Random_Forest_best_model.pkl


 TASK 5.5: EXPERIMENT TRACKING AND MODEL REGISTRY

In [50]:
# ============================================
# TASK 5.5: EXPERIMENT TRACKING AND MODEL REGISTRY
# ============================================

import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

EXPERIMENT_NAME = "Credit_Risk_Modeling_Task5"
mlflow.set_experiment(EXPERIMENT_NAME)

# Best models from hyperparameter tuning
best_models = {
    "Gradient_Boosting": tuned_gb_model,
    "Random_Forest": tuned_rf_model
}
for name, model in best_models.items():
    with mlflow.start_run(run_name=name):
        # Log the model in MLflow
        mlflow.sklearn.log_model(model, name="model_pipeline")
        
        # Evaluate on test set
        y_pred = model.predict(X_test_processed)
        y_proba = model.predict_proba(X_test_processed)[:,1] if hasattr(model, "predict_proba") else None
        
        # Metrics
        acc = accuracy_score(y_test, y_pred)
        roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
        
        # Log metrics
        mlflow.log_metric("accuracy", acc)
        if roc is not None:
            mlflow.log_metric("roc_auc", roc)
        
        # Print classification report safely
        roc_display = f"{roc:.3f}" if roc is not None else "N/A"
        print(f"\n{name} Test Accuracy: {acc:.3f}, ROC-AUC: {roc_display}")
        print(classification_report(y_test, y_pred, zero_division=0))
        
        # Register the model to MLflow Model Registry
        model_uri = f"runs:/{mlflow.active_run().info.run_id}/model_pipeline"
        mlflow.register_model(model_uri, f"Best_{name}_CreditRisk")
        print(f"✅ Registered {name} model in MLflow Model Registry\n")


Gradient_Boosting Test Accuracy: 0.613, ROC-AUC: 0.510
              precision    recall  f1-score   support

           0       0.64      0.91      0.75       468
           1       0.32      0.07      0.12       259

    accuracy                           0.61       727
   macro avg       0.48      0.49      0.44       727
weighted avg       0.52      0.61      0.53       727



Registered model 'Best_Gradient_Boosting_CreditRisk' already exists. Creating a new version of this model...
2025/12/18 09:18:25 WARNING mlflow.tracking._model_registry.fluent: Run with id 5a7d8400b0db4d9ca67ba8ab50504241 has no artifacts at artifact path 'model_pipeline', registering model based on models:/m-3d0ad266ee414b7ab166444002d57bb8 instead
Created version '3' of model 'Best_Gradient_Boosting_CreditRisk'.


✅ Registered Gradient_Boosting model in MLflow Model Registry


Random_Forest Test Accuracy: 0.549, ROC-AUC: 0.505
              precision    recall  f1-score   support

           0       0.65      0.64      0.65       468
           1       0.37      0.38      0.37       259

    accuracy                           0.55       727
   macro avg       0.51      0.51      0.51       727
weighted avg       0.55      0.55      0.55       727



Successfully registered model 'Best_Random_Forest_CreditRisk'.
2025/12/18 09:18:30 WARNING mlflow.tracking._model_registry.fluent: Run with id a9ac998ee0f14db2924c80c12467bcfe has no artifacts at artifact path 'model_pipeline', registering model based on models:/m-d9c38c0eacec4c00a2d8cb05ea0ba8e7 instead


✅ Registered Random_Forest model in MLflow Model Registry



Created version '1' of model 'Best_Random_Forest_CreditRisk'.


Task 5.6 – Register the best model in MLflow

In [53]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# Example: Wrap your model in a pipeline with preprocessing
gb_pipeline = Pipeline([
    ('preprocessor', preprocessor),       # the ColumnTransformer
    ('classifier', GradientBoostingClassifier(
        learning_rate=0.1, max_depth=3, 
        n_estimators=150, subsample=1.0,
        random_state=42))
])

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        max_depth=10, min_samples_split=2, 
        min_samples_leaf=1, n_estimators=100, 
        random_state=42))
])

# Train pipelines on training data
gb_pipeline.fit(X_train, y_train)
rf_pipeline.fit(X_train, y_train)

# Log the full pipelines to MLflow
import mlflow
import mlflow.sklearn

EXPERIMENT_NAME = "Credit_Risk_Modeling_Task5"
mlflow.set_experiment(EXPERIMENT_NAME)

pipelines = {"Gradient_Boosting": gb_pipeline, "Random_Forest": rf_pipeline}

for name, pipe in pipelines.items():
    with mlflow.start_run(run_name=name):
        mlflow.sklearn.log_model(pipe, "model_pipeline")
        print(f"✅ Logged pipeline {name} to MLflow")

2025/12/18 09:21:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/18 09:21:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Logged pipeline Gradient_Boosting to MLflow
✅ Logged pipeline Random_Forest to MLflow


TASK 5.6: Model Evaluation for Registered Model

In [60]:
# ============================================
# TASK 5.6: Save, Evaluate, and Register Full Pipelines
# ============================================

import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# --- Best full pipelines (preprocessor + model)
best_pipelines = {
    "Gradient_Boosting": gb_pipeline,  # include preprocessing
    "Random_Forest": rf_pipeline
}

EXPERIMENT_NAME = "Credit_Risk_Modeling_Task5"
mlflow.set_experiment(EXPERIMENT_NAME)

# ---------------------------
# Step 1: Log full pipelines to MLflow
# ---------------------------
logged_runs = {}  # store run_id for later
for name, pipeline in best_pipelines.items():
    with mlflow.start_run(run_name=name) as run:
        mlflow.sklearn.log_model(pipeline, "model_pipeline")  # full pipeline
        print(f"✅ Logged full pipeline {name} to MLflow")
        logged_runs[name] = run.info.run_id

# ---------------------------
# Step 2: Evaluate the pipelines (using raw X_test)
# ---------------------------
for name, run_id in logged_runs.items():
    model_uri = f"runs:/{run_id}/model_pipeline"
    loaded_pipeline = mlflow.sklearn.load_model(model_uri)

    # Predict on raw test set (pipeline handles preprocessing)
    y_pred = loaded_pipeline.predict(X_test)
    y_proba = loaded_pipeline.predict_proba(X_test)[:, 1] if hasattr(loaded_pipeline, "predict_proba") else None

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    roc_text = f"{roc:.3f}" if roc is not None else "N/A"

    print(f"\n{name} Test Accuracy: {acc:.3f}, ROC-AUC: {roc_text}")
    print(classification_report(y_test, y_pred, zero_division=0))

    # Log evaluation metrics back to MLflow
    with mlflow.start_run(run_name=f"Evaluation_{name}"):
        mlflow.log_metric("accuracy", acc)
        if roc is not None:
            mlflow.log_metric("roc_auc", roc)
        print(f"✅ Logged evaluation metrics for {name} to MLflow")

# ---------------------------
# Step 3: Register the best models
# ---------------------------
for name, run_id in logged_runs.items():
    model_uri = f"runs:/{run_id}/model_pipeline"
    model_registry_name = f"Best_{name}_CreditRisk"

    try:
        mlflow.register_model(model_uri, model_registry_name)
        print(f"✅ Registered {name} model in MLflow Model Registry")
    except mlflow.exceptions.MlflowException as e:
        print(f"⚠️ Model {name} may already exist. Created new version. Details: {str(e)}")

2025/12/18 09:27:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/18 09:27:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Logged full pipeline Gradient_Boosting to MLflow
✅ Logged full pipeline Random_Forest to MLflow

Gradient_Boosting Test Accuracy: 0.715, ROC-AUC: 0.781
              precision    recall  f1-score   support

           0       0.83      0.70      0.76       468
           1       0.58      0.74      0.65       259

    accuracy                           0.72       727
   macro avg       0.70      0.72      0.70       727
weighted avg       0.74      0.72      0.72       727

✅ Logged evaluation metrics for Gradient_Boosting to MLflow

Random_Forest Test Accuracy: 0.691, ROC-AUC: 0.762
              precision    recall  f1-score   support

           0       0.81      0.67      0.74       468
           1       0.55      0.72      0.62       259

    accuracy                           0.69       727
   macro avg       0.68      0.70      0.68       727
weighted avg       0.72      0.69      0.70       727

✅ Logged evaluation metrics for Random_Forest to MLflow


Registered model 'Best_Gradient_Boosting_CreditRisk' already exists. Creating a new version of this model...
2025/12/18 09:27:26 WARNING mlflow.tracking._model_registry.fluent: Run with id 1ab9f318dbaf4d0cb4e1ddc4549b52fa has no artifacts at artifact path 'model_pipeline', registering model based on models:/m-1aae25f60efd457fbe90228906cba74a instead
Created version '5' of model 'Best_Gradient_Boosting_CreditRisk'.
Registered model 'Best_Random_Forest_CreditRisk' already exists. Creating a new version of this model...


✅ Registered Gradient_Boosting model in MLflow Model Registry


2025/12/18 09:27:27 WARNING mlflow.tracking._model_registry.fluent: Run with id 5d01fbe11b904ba999418e453414af6a has no artifacts at artifact path 'model_pipeline', registering model based on models:/m-6dd8376f26e840c0ab3ebbbf3c9112dc instead


✅ Registered Random_Forest model in MLflow Model Registry


Created version '2' of model 'Best_Random_Forest_CreditRisk'.
